<a href="https://colab.research.google.com/github/rkhushii/Codsoft-ML-task-3/blob/main/main/Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from google.colab import drive
with open('/content/train_data.txt', 'r') as f:
  lines = f.readlines()

# Split each line into columns using the ::: separator
data = [line.split(':::') for line in lines]

# Create a dataframe from the list of columns
df = pd.DataFrame(data, columns=['column1', 'column2', 'column3','col4'])

# Print the dataframe
print(df)
x_train=df['col4'][:int(len(df)*0.4)]
y_train=df['column3'][:int(len(df)*0.4)]
print(x_train,y_train)


      column1                                            column2      column3  \
0          1                       Oscar et la dame rose (2009)        drama    
1          2                                       Cupid (1997)     thriller    
2          3                   Young, Wild and Wonderful (1980)        adult    
3          4                              The Secret Sin (1915)        drama    
4          5                             The Unrecovered (2007)        drama    
...       ...                                                ...          ...   
17547  17548    "The Dream Team with Annabelle and Michael" (...   talk-show    
17548  17549                        You Can't Hurry Love (2016)      romance    
17549  17550                              A Little Water (2017)        drama    
17550  17551                             Life of Josutty (2015)        drama    
17551  17552                                    Jump Cut (1993)       comedy    

                           

In [3]:
x_test=df['col4'][int(len(df)*0.2):]
y_test=df['column3'][int(len(df)*0.2):]

In [4]:
u=y_train.unique()

In [5]:
genre_dict={}
for i in range(len(u)):
  genre_dict[u[i]]=i

In [6]:
genre_dict

{' drama ': 0,
 ' thriller ': 1,
 ' adult ': 2,
 ' documentary ': 3,
 ' comedy ': 4,
 ' crime ': 5,
 ' reality-tv ': 6,
 ' horror ': 7,
 ' sport ': 8,
 ' animation ': 9,
 ' action ': 10,
 ' fantasy ': 11,
 ' short ': 12,
 ' sci-fi ': 13,
 ' music ': 14,
 ' adventure ': 15,
 ' talk-show ': 16,
 ' western ': 17,
 ' family ': 18,
 ' mystery ': 19,
 ' history ': 20,
 ' news ': 21,
 ' biography ': 22,
 ' romance ': 23,
 ' game-show ': 24,
 ' musical ': 25,
 ' war ': 26}

In [7]:
y_train.replace(to_replace=genre_dict, inplace=True)
y_train

0       0
1       1
2       2
3       0
4       0
       ..
7015    3
7016    0
7017    3
7018    0
7019    0
Name: column3, Length: 7020, dtype: int64

In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
vocab_dict={}
for message in x_train:
  for word in nltk.word_tokenize(message):
    if word not in vocab_dict:
      vocab_dict[word]=1
    else:
      vocab_dict[word]+=1


In [10]:
VOCAB_SIZE = len(vocab_dict)
MAX_LENGTH = len(max(x_train, key=lambda p: len(p.split())).split())
print(VOCAB_SIZE,MAX_LENGTH)

52996 1829


In [11]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import one_hot
from keras.utils import pad_sequences

In [12]:
encoded_text = [one_hot(d, VOCAB_SIZE) for d in x_train]
padded_text = pad_sequences(encoded_text, maxlen=MAX_LENGTH, padding='post')

In [13]:
padded_text

array([[16748,   426, 17789, ...,     0,     0,     0],
       [49386, 36035,  7182, ...,     0,     0,     0],
       [25749, 42005, 31539, ...,     0,     0,     0],
       ...,
       [18962, 17789, 42005, ...,     0,     0,     0],
       [49386, 37700, 14506, ...,     0,     0,     0],
       [ 8022, 10520,  7182, ...,     0,     0,     0]], dtype=int32)

In [14]:
genre_classifier_model= tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=VOCAB_SIZE,
        output_dim= 50,mask_zero=True),
    #tf.keras.layers.Flatten(),
    tf.keras.layers.LSTM(64),
   # tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(28,activation='softmax')
])

In [15]:
genre_classifier_model.compile(loss='sparse_categorical_crossentropy', metrics=['acc'], optimizer='adam')

In [16]:
import numpy as np

In [17]:
genre_classifier_model.fit(np.array(padded_text),np.array( y_train),batch_size=64,epochs=15)

Epoch 1/15
110/110 [==============================] - 120s 1s/step - loss: 2.5325 - acc: 0.2363
Epoch 2/15
110/110 [==============================] - 114s 1s/step - loss: 2.3368 - acc: 0.2856
Epoch 3/15
110/110 [==============================] - 114s 1s/step - loss: 2.1053 - acc: 0.4073
Epoch 4/15
110/110 [==============================] - 114s 1s/step - loss: 1.7648 - acc: 0.5127
Epoch 5/15
110/110 [==============================] - 114s 1s/step - loss: 1.4568 - acc: 0.5960
Epoch 6/15
110/110 [==============================] - 114s 1s/step - loss: 1.2033 - acc: 0.6684
Epoch 7/15
110/110 [==============================] - 114s 1s/step - loss: 0.9843 - acc: 0.7266
Epoch 8/15
110/110 [==============================] - 113s 1s/step - loss: 0.7932 - acc: 0.7815
Epoch 9/15
110/110 [==============================] - 113s 1s/step - loss: 0.6538 - acc: 0.8201
Epoch 10/15
 77/110 [====================>.........] - ETA: 33s - loss: 0.5344 - acc: 0.8580

KeyboardInterrupt: 

In [ ]:
def predict_message(movie_description_text):
    prediction=[]
    padded_sample = pad_sequences([one_hot(movie_description_text, VOCAB_SIZE)], maxlen=MAX_LENGTH, padding='post')
    sample_result=genre_classifier_model.predict(np.array(padded_sample))
    max_index = np.argmax(sample_result)
    print("Predicted genre: ",u[max_index])

In [ ]:
predict_message("His father has died, he hasn't spoken with his brother for about 10 years and has a serious cancer. Diego is a talented film director with difficulty to deal with his sickness, which is making him lose his friends and family. His best friend and doctor Ricardo gives him the news that he needs a bone marrow transplantation, otherwise he'll die. He gets married to a beautiful woman, Livia, just before going to Seattle to get treatment. There, he undergoes numerous medical procedures. During treatment, he meets an Hindu boy, with whom he plays and whom he tells amazing stories. Odds are against him and when stakes are the highest, Diego gets a visit from a very uncommon man.")